In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dubai Satellite Imagery Semantic Segmentation
Humans in the Loop has published an open access dataset annotated for a joint project with the Mohammed Bin Rashid Space Center in Dubai, the UAE.

The dataset consists of aerial imagery of Dubai obtained by MBRSC satellites and annotated with pixel-wise semantic segmentation in 6 classes. The images were segmented by the trainees of the Roia Foundation in Syria.

Original Dataset Link: https://humansintheloop.org/resources/datasets/semantic-segmentation-dataset/

# Installing & Importing Libraries

In [ ]:
import pickle
import numpy as np
import pandas as pd
from PIL import Image
import albumentations as A
from IPython.display import SVG
import matplotlib.pyplot as plt
%matplotlib inline
import os, re, sys, random, shutil, cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras import applications, optimizers
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import model_to_dot, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger, LearningRateScheduler
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, ZeroPadding2D, Dropout

In [ ]:
def visualize(image, mask, original_image=None, original_mask=None):
    fontsize = 16

    if original_image is None and original_mask is None:
        f, ax = plt.subplots(2, 1, figsize=(10, 10), squeeze=True)
        f.set_tight_layout(h_pad=5, w_pad=5)

        ax[0].imshow(image)
        ax[1].imshow(mask)
    else:
        f, ax = plt.subplots(2, 2, figsize=(16, 12), squeeze=True)
        plt.tight_layout(pad=0.2, w_pad=1.0, h_pad=0.01)

        ax[0, 0].imshow(original_image)
        ax[0, 0].set_title('Original Image', fontsize=fontsize)

        ax[1, 0].imshow(original_mask)
        ax[1, 0].set_title('Original Mask', fontsize=fontsize)

        ax[0, 1].imshow(image)
        ax[0, 1].set_title('Transformed Image', fontsize=fontsize)

        ax[1, 1].imshow(mask)
        ax[1, 1].set_title('Transformed Mask', fontsize=fontsize)

    plt.savefig('sample_augmented_image.png', facecolor= 'w', transparent= False, bbox_inches= 'tight', dpi= 100)

# Working with Augmented Dataset

In [ ]:
train_images = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Images/Train/"
train_masks = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Masks/Train/"
val_images = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Images/Validation/"
val_masks = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Masks/Validation/"

In [ ]:
file_names = np.sort(os.listdir(train_images ))
file_names = np.char.split(file_names, '.')
filenames = np.array([])
for i in range(len(file_names)):
    filenames = np.append(filenames, file_names[i][0])

In [ ]:
def show_data(files, original_images_dir, label_images_dir):

    for file in files:
        fig, axs = plt.subplots(1, 2, figsize=(15, 6), constrained_layout=True)

        axs[0].imshow(cv2.resize(cv2.imread(original_images_dir+str(file)+'.jpg'), (2000,1400)))
        axs[0].set_title('Original Image', fontdict = {'fontsize':14, 'fontweight': 'medium'})
        axs[0].set_xticks(np.arange(0, 2001, 200))
        axs[0].set_yticks(np.arange(0, 1401, 200))
        axs[0].grid(False)
        axs[0].axis(True)

        semantic_label_image = cv2.imread(label_images_dir+str(file)+'.png')
       # semantic_label_image = cv2.cvtColor(semantic_label_image, cv2.COLOR_BGR2RGB)
        semantic_label_image = cv2.resize(semantic_label_image, (2000,1400))
        axs[1].imshow(semantic_label_image)
        axs[1].set_title('Semantic Segmentation Mask', fontdict = {'fontsize':14, 'fontweight': 'medium'})
        axs[1].set_xticks(np.arange(0, 2001, 200))
        axs[1].set_yticks(np.arange(0, 1401, 200))
        axs[1].grid(False)
        axs[1].axis(True)

        plt.savefig('./sample_'+file, facecolor= 'w', transparent= False, bbox_inches= 'tight', dpi= 100)
        plt.show()



#files = ['patch_5454', 'patch_5564', 'patch_5741', 'patch_5965', 'patch_6066', 'patch_6955', 'patch_7259',]
#show_data(files, train_images, train_masks)

In [ ]:
import pandas as pd

# Define the class_dict data as a list of dictionaries
class_dict_data = [
    {'name': 'building', 'r': 60, 'g': 16, 'b': 152},
    {'name': 'land', 'r': 132, 'g': 41, 'b': 246},
    {'name': 'road', 'r': 110, 'g': 193, 'b': 228},
    {'name': 'vegetation', 'r': 254, 'g': 221, 'b': 58},
    {'name': 'water', 'r': 226, 'g': 169, 'b': 41},
    {'name': 'unlabeled', 'r': 155, 'g': 155, 'b': 155}
]

# Create the DataFrame
class_dict_df = pd.DataFrame(class_dict_data)
class_dict_df
# Now, class_dict_df contains the desired DataFrame

,name,r,g,b
0,building,60,16,152
1,land,132,41,246
2,road,110,193,228
3,vegetation,254,221,58
4,water,226,169,41
5,unlabeled,155,155,155


In [ ]:
label_names= list(class_dict_df.name)
label_codes = []
r= np.asarray(class_dict_df.r)
g= np.asarray(class_dict_df.g)
b= np.asarray(class_dict_df.b)

for i in range(len(class_dict_df)):
    label_codes.append(tuple([r[i], g[i], b[i]]))

label_codes, label_names

([(60, 16, 152),
  (132, 41, 246),
  (110, 193, 228),
  (254, 221, 58),
  (226, 169, 41),
  (155, 155, 155)],
 ['building', 'land', 'road', 'vegetation', 'water', 'unlabeled'])

# Create Useful Label & Code Conversion Dictionaries

These will be used for:

* One hot encoding the mask labels for model training
* Decoding the predicted labels for interpretation and visualization

In [ ]:
code2id = {v:k for k,v in enumerate(label_codes)}
id2code = {k:v for k,v in enumerate(label_codes)}

name2id = {v:k for k,v in enumerate(label_names)}
id2name = {k:v for k,v in enumerate(label_names)}

In [ ]:
id2code

{0: (60, 16, 152),
 1: (132, 41, 246),
 2: (110, 193, 228),
 3: (254, 221, 58),
 4: (226, 169, 41),
 5: (155, 155, 155)}

In [ ]:
id2name

{0: 'building',
 1: 'land',
 2: 'road',
 3: 'vegetation',
 4: 'water',
 5: 'unlabeled'}

# Define Functions for One Hot Encoding RGB Labels & Decoding Encoded Predictions

In [ ]:
def rgb_to_onehot(rgb_image, colormap = id2code):
    '''Function to one hot encode RGB mask labels
        Inputs:
            rgb_image - image matrix (eg. 256 x 256 x 3 dimension numpy ndarray)
            colormap - dictionary of color to label id
        Output: One hot encoded image of dimensions (height x width x num_classes) where num_classes = len(colormap)
    '''
    num_classes = len(colormap)
    shape = rgb_image.shape[:2]+(num_classes,)
    encoded_image = np.zeros( shape, dtype=np.int8 )
    for i, cls in enumerate(colormap):
        encoded_image[:,:,i] = np.all(rgb_image.reshape( (-1,3) ) == colormap[i], axis=1).reshape(shape[:2])
    return encoded_image


def onehot_to_rgb(onehot, colormap = id2code):
    '''Function to decode encoded mask labels
        Inputs:
            onehot - one hot encoded image matrix (height x width x num_classes)
            colormap - dictionary of color to label id
        Output: Decoded RGB image (height x width x 3)
    '''
    single_layer = np.argmax(onehot, axis=-1)
    output = np.zeros( onehot.shape[:2]+(3,) )
    for k in colormap.keys():
        output[single_layer==k] = colormap[k]
    return np.uint8(output)

# Creating Custom Image Data Generators
## Defining Data Generators

# Custom Image Data Generators for Creating Batches of Frames and Masks

# Model

In [ ]:
train_images = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Images/Train/"
train_masks = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Masks/Train/"
val_images = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Images/Validation/"
val_masks = "/content/drive/MyDrive/Colab Notebooks/datasets/Aug/Masks/Validation/"


In [ ]:
from keras.utils import to_categorical

# List all image and mask files for training and validation
train_image_files = os.listdir(train_images)
train_mask_files = os.listdir(train_masks)
val_image_files = os.listdir(val_images)
val_mask_files = os.listdir(val_masks)

# Define a generator to load data batch by batch
def data_generator(image_files, mask_files, batch_size, num_classes, colormap):
    num_samples = len(image_files)
    while True:
        batch_indices = np.random.choice(num_samples, batch_size, replace=False)
        batch_images = []
        batch_masks = []
        for index in batch_indices:
            image = cv2.imread(os.path.join(train_images, image_files[index]))
            mask = cv2.imread(os.path.join(train_masks, mask_files[index]), cv2.IMREAD_COLOR)

            # Preprocess and augment data as needed

            # One-hot encode the mask using the provided functions
            mask_onehot = rgb_to_onehot(mask, colormap)

            # Add data to batch_images and batch_masks
            batch_images.append(image)
            batch_masks.append(mask_onehot)

        yield np.array(batch_images), np.array(batch_masks)
# Define a generator to load data batch by batch
def data_generator2(image_files, mask_files, batch_size, num_classes, colormap):
    num_samples = len(image_files)
    while True:
        batch_indices = np.random.choice(num_samples, batch_size, replace=False)
        batch_images = []
        batch_masks = []
        for index in batch_indices:
            image = cv2.imread(os.path.join(val_images, image_files[index]))
            mask = cv2.imread(os.path.join(val_masks, mask_files[index]), cv2.IMREAD_COLOR)

            # Preprocess and augment data as needed

            # One-hot encode the mask using the provided functions
            mask_onehot = rgb_to_onehot(mask, colormap)

            # Add data to batch_images and batch_masks
            batch_images.append(image)
            batch_masks.append(mask_onehot)

        yield np.array(batch_images), np.array(batch_masks)


input_shape = (256, 256, 3)  # Adjust the input shape based on your images
num_classes = len(id2code)  # Make sure to define your colormap




In [ ]:
batch_size = 16
num_train_samples = len(np.sort(os.listdir(train_images)))
num_val_samples = len(np.sort(os.listdir(val_images)))
steps_per_epoch = np.ceil(float(num_train_samples) / float(2*batch_size))
print('steps_per_epoch: ', steps_per_epoch)
validation_steps = np.ceil(float(num_val_samples) / float(2*batch_size))
print('validation_steps: ', validation_steps)

steps_per_epoch:  160.0
validation_steps:  35.0
